In [1]:
import pandas as pd

from scripts.downloader import DownloaderAktualne, DownloaderIdnes

Succesfully initialized Downloader.


In [2]:
# Config
url_aktualne = "https://zpravy.aktualne.cz/domaci/"
url_idnes = "https://www.idnes.cz/zpravy/domaci/"

data_aktualne = "data/articles_Aktualne.cz.csv"
data_idnes = "data/articles_iDnes.cz.csv"

In [3]:
# Init verbose DownloaderAktualne and DownloaderIdnes
d_aktualne = DownloaderAktualne(url_aktualne, save_to=data_aktualne, verbose=True)
d_idnes = DownloaderIdnes(url_idnes, save_to=data_idnes, verbose=True)

Succesfully initialized Downloader.
Succesfully initialized Downloader.


In [4]:
# Download demo
d_aktualne.downloadHeadlines()
d_aktualne.saveToCsv()

d_idnes.downloadHeadlines()
d_idnes.saveToCsv()

In [8]:
articles = pd.read_csv(data_idnes)

In [9]:
articles

,article_id,slug,date,time,is_updated,headline,excerpt,article_url,scraped_at
0,A200503_114708_domaci_rejs,videa-tydne-nevera-moderator-pentagon-ufo-skla...,NaN,NaN,False,"VIDEA TÝDNE: Nevěra v přímém přenosu, nazí zah...","Na velký malér si zadělal španělský moderátor,...",https://www.idnes.cz/zpravy/domaci/videa-tydne...,03-05-2020 17:44:48
1,A200428_111910_domaci_lre,rozhovor-cestina-pravdova-akademie-ved-koronav...,NaN,NaN,False,Koronáčelník nás zaprymuloval do harantény. O ...,Pandemie koronaviru ovlivnila nejen naše život...,https://www.idnes.cz/zpravy/domaci/rozhovor-ce...,03-05-2020 17:44:48
2,A200430_112440_domaci_rik,lenin-hitler-komunismus-nacismus,NaN,NaN,False,Alternativní historie bez Hitlera a Lenina. Dě...,"Je to pár dní, co uplynulo 150 let od narození...",https://www.idnes.cz/zpravy/domaci/lenin-hitle...,03-05-2020 17:44:48
3,A200429_175133_domaci_lre,koronavirus-slovni-zasoba-nova-slova-korona-ko...,NaN,NaN,False,"KVÍZ: Víte, co je to koronáč nebo prdom? Otest...","Co je to prymulex, koronáčelník nebo v současn...",https://www.idnes.cz/zpravy/domaci/koronavirus...,03-05-2020 17:44:48
4,A200416_165739_domaci_aug,hydrolog-bohumir-jansky-zemedelstvi-zmena-reky...,NaN,NaN,False,"Nebýt staveb našich předků, boj se suchem jsme...",Hydrolog Bohumír Janský z Přírodovědecké fakul...,https://www.idnes.cz/zpravy/domaci/hydrolog-bo...,03-05-2020 17:44:48
...,...,...,...,...,...,...,...,...,...
103,A200428_133040_vary-zpravy_ba,valdstejn-kun-exponat-muzeum-cheb-stehovani-va...,2020-04-28T15:35:00,2020-04-28T15:35:00,False,Po lomených schodech v muzeu stěhovalo šest mu...,"Výjimečný kůň Albrechta z Valdštejna, který se...",https://www.idnes.cz/karlovy-vary/zpravy/valds...,03-05-2020 17:44:53
104,A200428_100622_domaci_lre,nemocnice-na-homolce-odmeny-pro-sestry-zadne-n...,2020-04-28T14:54:00,2020-04-28T14:54:00,False,"Snížíme odměny, napsal překvapeným zdravotníků...","V nemocnicích tráví dlouhé hodiny, zachraňují ...",https://www.idnes.cz/zpravy/domaci/nemocnice-n...,03-05-2020 17:44:53
105,A200428_142241_domaci_remy,koronavirus-nouzovy-stav-hrozi-druha-vlna-epid...,2020-04-28T14:28:00,2020-04-28T14:28:00,False,"Bez stavu nouze hrozí druhá vlna epidemie, var...",Bez prodloužení nouzového stavu by podle praco...,https://www.idnes.cz/zpravy/domaci/koronavirus...,03-05-2020 17:44:53
106,A200428_130400_domaci_blj,umrti-filozof-ladislav-hejdanek-92-let-disident,2020-04-28T13:32:00,2020-04-28T13:32:00,False,"Zemřel filozof Ladislav Hejdánek, někdejšímu d...","Zemřel filozof, publicista a někdejší mluvčí C...",https://www.idnes.cz/zpravy/domaci/umrti-filoz...,03-05-2020 17:44:53
